## Apiux & SII: calculo de indice de materia oscura en personas juridicas.
## ATENCION: proyecto sujeto a mantenimiento continuo. 

## Henry Vega (henrry.vega@api-ux.com)
## Data analyst

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
import pyspark
#warnings.filterwarnings('ignore', category=DeprecationWarning)

In [2]:

spark = SparkSession.builder \
  .appName("Test")  \
  .config("spark.yarn.access.hadoopFileSystems","abfs://data@datalakesii.dfs.core.windows.net/") \
  .config("spark.executor.memory", "24g") \
  .config("spark.driver.memory", "12g")\
  .config("spark.executor.cores", "12") \
  .config("spark.executor.instances", "24") \
  .config("spark.driver.maxResultSize", "12g") \
  .getOrCreate()

Setting spark.hadoop.yarn.resourcemanager.principal to yasser.nanjari


## Carga de relaciones societarias(depurada)

Se carga la data depurada anteriormente de relaciones societarias.

In [3]:
df = spark.read.options(header=True,inferSchema=True,delimiter=",").csv("../../data/processed/sociedades_participacion_capital_nozero.csv")
df.createOrReplaceTempView("sociedad")
spark.sql("select * from sociedad order by RUT_SOCIEDAD asc").show()

Hive Session ID = e3bd61a8-8f35-46bc-9515-47f8158d8739                          
[Stage 2:===================>                                      (4 + 8) / 12]

+--------------------+--------------------+------------------+---------------------+
|        RUT_SOCIEDAD|           RUT_SOCIO|PORCENTAJE_CAPITAL|PORCENTAJE_UTILIDADES|
+--------------------+--------------------+------------------+---------------------+
|+++1gCGSMhlEHLJDQ...|jBIDEmi35uF/bAHr2...|             100.0|                100.0|
|+++40fb1kG88W0AEf...|bmXOLtRFjJPrLjKzQ...|             100.0|                100.0|
|+++aS3F42zAfvw7QX...|WdafNmzolsFbUp2HA...|             100.0|                100.0|
|++/7Y0IzKUdH2J8C/...|41WRUY90glpRrq9XI...|             100.0|                100.0|
|++/p+MzwTKR1XVz6S...|+bJxBrWdS6tCHMPtN...|             100.0|                100.0|
|++00DcdTZAhzp008i...|aL6LEEk8P6vB8PyHu...|             100.0|                100.0|
|++0Cw0fQKv+4UABY2...|eJvYBKdvqhJ6I1vK5...|             100.0|                100.0|
|++0PaA9Gk1ZMElZZL...|VkY6uutbGR+7iTLUS...|              50.0|                 50.0|
|++0PaA9Gk1ZMElZZL...|Y20GxtSITZxJGWolh...|              50.0|   

In [4]:
spark.sql("select * from sociedad where RUT_SOCIEDAD like 'ZGZPH%'").show()
spark.sql("select * from sociedad where RUT_SOCIEDAD like 'XYtA%'").show()
spark.sql("select * from sociedad where RUT_SOCIEDAD like '2EJB%'").show()
spark.sql("select * from sociedad where RUT_SOCIEDAD like 'ySz0%'").show()
spark.sql("select * from sociedad where RUT_SOCIEDAD like 'LhJi%'").show()
spark.sql("select * from sociedad where RUT_SOCIEDAD like 'POAS%'").show()
spark.sql("select * from sociedad where RUT_SOCIEDAD like 'RAuy%'").show()
spark.sql("select * from sociedad where RUT_SOCIEDAD like 'J+WR%'").show()

+------------+---------+------------------+---------------------+
|RUT_SOCIEDAD|RUT_SOCIO|PORCENTAJE_CAPITAL|PORCENTAJE_UTILIDADES|
+------------+---------+------------------+---------------------+
+------------+---------+------------------+---------------------+



+------------+---------+------------------+---------------------+
|RUT_SOCIEDAD|RUT_SOCIO|PORCENTAJE_CAPITAL|PORCENTAJE_UTILIDADES|
+------------+---------+------------------+---------------------+
+------------+---------+------------------+---------------------+



+------------+---------+------------------+---------------------+
|RUT_SOCIEDAD|RUT_SOCIO|PORCENTAJE_CAPITAL|PORCENTAJE_UTILIDADES|
+------------+---------+------------------+---------------------+
+------------+---------+------------------+---------------------+

+------------+---------+------------------+---------------------+
|RUT_SOCIEDAD|RUT_SOCIO|PORCENTAJE_CAPITAL|PORCENTAJE_UTILIDADES|
+------------+---------+------------------+---------------------+
+------------+---------+------------------+---------------------+

+------------+---------+------------------+---------------------+
|RUT_SOCIEDAD|RUT_SOCIO|PORCENTAJE_CAPITAL|PORCENTAJE_UTILIDADES|
+------------+---------+------------------+---------------------+
+------------+---------+------------------+---------------------+

+------------+---------+------------------+---------------------+
|RUT_SOCIEDAD|RUT_SOCIO|PORCENTAJE_CAPITAL|PORCENTAJE_UTILIDADES|
+------------+---------+------------------+---------------------+
+------

+------------+---------+------------------+---------------------+
|RUT_SOCIEDAD|RUT_SOCIO|PORCENTAJE_CAPITAL|PORCENTAJE_UTILIDADES|
+------------+---------+------------------+---------------------+
+------------+---------+------------------+---------------------+



[Stage 26:========>                                                 (1 + 6) / 7]

+------------+---------+------------------+---------------------+
|RUT_SOCIEDAD|RUT_SOCIO|PORCENTAJE_CAPITAL|PORCENTAJE_UTILIDADES|
+------------+---------+------------------+---------------------+
+------------+---------+------------------+---------------------+



## Exploracion de composiciones de socios en capital y utilidades

Como no hay unicidad de entradas y se puede presentar varias veces una combinacion sociedad y socio, se agrupa por el promedio para cada relacion. De esta forma analizamos la composicion de la sociedad en utilidades y capital.


In [5]:
spark.sql("select  RUT_SOCIEDAD, RUT_SOCIO, mean(PORCENTAJE_CAPITAL) as PORCENTAJE_CAPITAL, mean(PORCENTAJE_UTILIDADES) as PORCENTAJE_UTILIDADES from sociedad group by RUT_SOCIEDAD, RUT_SOCIO").createOrReplaceTempView("composicion")
spark.sql("select  RUT_SOCIEDAD, sum(PORCENTAJE_CAPITAL) as TOTAL_CAPITAL, SUM(PORCENTAJE_UTILIDADES) as TOTAL_UTILIDADES from composicion group by RUT_SOCIEDAD").createOrReplaceTempView("composicion")
m=spark.sql("select * from composicion").toPandas()


Veamos cuantas sociedades suman mas del 100% en capital total.

In [6]:
m[m['TOTAL_CAPITAL']>105] 

,RUT_SOCIEDAD,TOTAL_CAPITAL,TOTAL_UTILIDADES
6625,4ycAGBELEjW5Sts2godBtQ==,110.00,100.00
56047,CbMOL0R9Zk9buhagFxIeXQ==,109.89,100.00
78847,Itw/jYPKxs9UYi1z4/xuDA==,108.90,108.90
95525,K8yZXhQKL+MpUpM0PrJlHw==,109.00,109.00
99314,EqLVYjdaxkTliFuxD0eskg==,110.00,110.00
...,...,...,...
1257508,ClIavBB7rp3t0XS2qZ20nA==,110.00,110.00
1271011,PuWDXUwj8YPMl+qyQYefYQ==,109.33,109.33
1283467,lHYHHHAaPsht2o+W1tMdBQ==,105.98,105.98
1296954,mdkrfI49Mf0IIlM2QhpJJw==,110.00,110.00


Veamos cuantas sociedades suman menos de 90% en capital total.

In [7]:
m[m['TOTAL_CAPITAL']<90] 

,RUT_SOCIEDAD,TOTAL_CAPITAL,TOTAL_UTILIDADES
630363,gwbsyFNCMsaytOcg8ftsKw==,11.29,11.29


In [8]:
m[m['TOTAL_UTILIDADES']>105] 

,RUT_SOCIEDAD,TOTAL_CAPITAL,TOTAL_UTILIDADES
852,zcnLnrlW8DVjOwN0gvUZDA==,100.00,109.00
9977,8bwCL0d00NOKo9MMOf5ebw==,100.00,190.00
13950,AshhiDXqEWdZLt/5SGi2eQ==,100.00,199.00
28651,EYUJqrSBL9GW6Ao08PNeyA==,100.00,180.00
28951,dX9WqqcHetWILGVKKhrt8Q==,100.00,109.00
...,...,...,...
1275903,bHpajn4n5RLFtn20YfLSEg==,100.00,110.00
1283467,lHYHHHAaPsht2o+W1tMdBQ==,105.98,105.98
1286164,m2RHlEGEFm1J0qF8ZN41ew==,100.00,109.00
1296954,mdkrfI49Mf0IIlM2QhpJJw==,110.00,110.00


In [9]:
m[m['TOTAL_UTILIDADES']<95] 

,RUT_SOCIEDAD,TOTAL_CAPITAL,TOTAL_UTILIDADES
402,QHlfqsVnl5ihsUAWltqFSQ==,90.0,90.00
527,qdX6ZMgpzraSoR0slCeDnQ==,94.0,94.00
1399,2PBBtganqeOh5mRV7plpZA==,100.0,90.00
1477,Q06LoGlhxme+Fr6tvi4GfA==,100.0,66.68
2762,wCnahH6I6fTGawvYKqXz7Q==,100.0,66.67
...,...,...,...
1312253,SgIbcfzeu4+2wRbiln48lA==,90.0,90.00
1313022,bhOcqWSeh5/bo3GsKOinpQ==,100.0,94.83
1313161,6OnR4ZbS6BY6xp62uIyQYg==,100.0,20.00
1313608,1W8BRzKJ47idGrxq/eKa4g==,100.0,60.00


Se utilizara el porcentaje de capital en lugar de utilidades para hacer la transmision de materia oscura. En el proceso de limpieza de datos se considero las filas con valores no nuels de PORCENTAJE_CAPITAL. Se utilizara una tabla de composicion para poder normalizar los valores de oscuridad para cuando los valores de porcentajes se capital sean diferentes al 100%.

## Tabla de ponderacion para el total de capital.

In [10]:
spark.sql("select RUT_SOCIEDAD as CONT_RUT, TOTAL_CAPITAL as ponderador from composicion").createOrReplaceTempView("composicion")

## Lectura de tabla de oscuridad

Se hace lectura de los datos de iniciales de oscuridad para personas naturales.

In [11]:
oscuridad1=spark.sql("select * from libsdf.jba_mat_obs_renta_1_e")
oscuridad1.createOrReplaceTempView("oscuridad")


In [12]:
spark.sql("select * from oscuridad where CONT_RUT LIKE 'eK5K%'").show()
spark.sql("select * from oscuridad where CONT_RUT LIKE '356d%'").show()
spark.sql("select * from oscuridad where CONT_RUT LIKE 'H1YZ%' ").show()
spark.sql("select * from oscuridad where CONT_RUT LIKE 'NUQp%' ").show()
spark.sql("select * from oscuridad where CONT_RUT LIKE 'U+ILU%' ").show()
spark.sql("select * from oscuridad where CONT_RUT LIKE 'Z923%' ").show()
spark.sql("select * from oscuridad where CONT_RUT LIKE 'XRHl%' ").show()

+--------------------+-------+-----+---------+---------+---------+---------+---------+---------+
|            CONT_RUT|CONT_DV|Valor|Periodo_6|Periodo_5|Periodo_4|Periodo_3|Periodo_2|Periodo_1|
+--------------------+-------+-----+---------+---------+---------+---------+---------+---------+
|eK5KCE9n7ek2CyHGi...|      7|  0.0|        2|        1|        2|        2|        2|        1|
|eK5K/z5V/MPOwfQjt...|      7|  1.0|     null|     null|     null|     null|     null|     null|
+--------------------+-------+-----+---------+---------+---------+---------+---------+---------+



+--------------------+-------+-----+---------+---------+---------+---------+---------+---------+
|            CONT_RUT|CONT_DV|Valor|Periodo_6|Periodo_5|Periodo_4|Periodo_3|Periodo_2|Periodo_1|
+--------------------+-------+-----+---------+---------+---------+---------+---------+---------+
|356dRC3qxWoTMyv8i...|      4|  0.0|        5|        5|        4|        2|        3|        1|
+--------------------+-------+-----+---------+---------+---------+---------+---------+---------+



+--------------------+-------+-----+---------+---------+---------+---------+---------+---------+
|            CONT_RUT|CONT_DV|Valor|Periodo_6|Periodo_5|Periodo_4|Periodo_3|Periodo_2|Periodo_1|
+--------------------+-------+-----+---------+---------+---------+---------+---------+---------+
|H1YZ+aH3OEsOrKPP3...|      1|  0.0|        1|        1|        2|        2|        2|        1|
|H1YZnbNvu/UAaou/X...|      4|  1.0|     null|     null|     null|     null|     null|     null|
|H1YZe/TuLYLl15YXC...|      3|  0.0|        3|        4|        4|        3|        3|        3|
+--------------------+-------+-----+---------+---------+---------+---------+---------+---------+



+--------------------+-------+------------+---------+---------+---------+---------+---------+---------+
|            CONT_RUT|CONT_DV|       Valor|Periodo_6|Periodo_5|Periodo_4|Periodo_3|Periodo_2|Periodo_1|
+--------------------+-------+------------+---------+---------+---------+---------+---------+---------+
|NUQpOUriivmj+Lu6M...|      3|         0.0|        2|        1|        2|        2|        1|        1|
|NUQpxKLFPsQ9NAywe...|      5|0.2215526409|     null|     null|        2|        2|        2|        1|
|NUQpDM2DVxOP53ZQ0...|      4|         0.0|        2|        4|        7|        6|        6|        5|
+--------------------+-------+------------+---------+---------+---------+---------+---------+---------+



+--------------------+-------+-----+---------+---------+---------+---------+---------+---------+
|            CONT_RUT|CONT_DV|Valor|Periodo_6|Periodo_5|Periodo_4|Periodo_3|Periodo_2|Periodo_1|
+--------------------+-------+-----+---------+---------+---------+---------+---------+---------+
|U+ILUZTnVmep3ZVHT...|      3|  0.0|        2|        3|        3|        3|        3|        3|
|U+ILUmC/fuqzGkZG+...|      7|  0.0|        4|        5|        6|        7|        7|        4|
+--------------------+-------+-----+---------+---------+---------+---------+---------+---------+



+--------------------+-------+------------+---------+---------+---------+---------+---------+---------+
|            CONT_RUT|CONT_DV|       Valor|Periodo_6|Periodo_5|Periodo_4|Periodo_3|Periodo_2|Periodo_1|
+--------------------+-------+------------+---------+---------+---------+---------+---------+---------+
|Z923n6TTDwHhay2No...|      0|         0.0|        1|        1|        1|        1|        1|        1|
|Z923rWI3WiFZDgJKa...|      8|0.1007057459|     null|        1|        1|        1|        1|        1|
|Z923KiXl6tBtmulKM...|      2|         0.0|        3|        2|        3|        2|        3|        2|
+--------------------+-------+------------+---------+---------+---------+---------+---------+---------+



[Stage 56:======================================================> (29 + 1) / 30]

+--------------------+-------+------------+---------+---------+---------+---------+---------+---------+
|            CONT_RUT|CONT_DV|       Valor|Periodo_6|Periodo_5|Periodo_4|Periodo_3|Periodo_2|Periodo_1|
+--------------------+-------+------------+---------+---------+---------+---------+---------+---------+
|XRHlFZBuQsZ4ujy82...|      8|0.8992942541|        3|     null|     null|     null|     null|     null|
|XRHlDRkjm/HXiRhaM...|      8|         0.0|        3|        3|        5|        5|        5|        3|
+--------------------+-------+------------+---------+---------+---------+---------+---------+---------+



## Ajuste de valores de participacion societaria.

Como para una combinacion sociedad socio tenemos distintos valores de PORCENTAJE_CAPITAL y PORCENTAJE_UTILIDADES se usara el promedio de dichos valores.

In [13]:
spark.sql("select  RUT_SOCIEDAD, RUT_SOCIO, mean(PORCENTAJE_CAPITAL) as PORCENTAJE_CAPITAL, mean(PORCENTAJE_UTILIDADES) as PORCENTAJE_UTILIDADES from sociedad group by RUT_SOCIEDAD, RUT_SOCIO").createOrReplaceTempView("sociedad")

## Primera iteracion

Se realiza el cruce de la data societaria con la data de  oscuridad de personas naturales (1re paso de calculo de materia oscura para sociedades completas). Para ello se completa la malla con la oscuridad inicial. Hay sociedades donde no se completan todos los socios con oscuridad, por lo cual se discrimina mediante un contador de nulos de dicho campo para poder agregar la data y obtener un valor de oscuridad para un rut de soiedad donde todas las entradas de sus socios han sido completadas.

In [14]:
#Iteracion 0
spark.sql("select * from sociedad left join oscuridad on sociedad.RUT_SOCIO=oscuridad.CONT_RUT order by sociedad.RUT_SOCIEDAD asc").createOrReplaceTempView("sociedad")
spark.sql("select * from sociedad ").show()
spark.sql("select RUT_SOCIEDAD, RUT_SOCIO, PORCENTAJE_CAPITAL, Valor from sociedad").createOrReplaceTempView("sociedad")
spark.sql("select * from sociedad ").show()
spark.sql("select RUT_SOCIEDAD as RUT_SOCIEDAD1, count(*) as nulos  from sociedad where Valor is null group by RUT_SOCIEDAD order by RUT_SOCIEDAD ASC").createOrReplaceTempView("aux")
spark.sql("select * from aux ").show()
spark.sql("select RUT_SOCIEDAD,RUT_SOCIO,PORCENTAJE_CAPITAL, nulos, Valor from sociedad left join aux on sociedad.RUT_SOCIEDAD=aux.RUT_SOCIEDAD1 order by RUT_SOCIEDAD asc ").createOrReplaceTempView("aux")
spark.sql("select * from aux where nulos is null ").show()


+--------------------+--------------------+------------------+---------------------+--------------------+-------+------------+---------+---------+---------+---------+---------+---------+
|        RUT_SOCIEDAD|           RUT_SOCIO|PORCENTAJE_CAPITAL|PORCENTAJE_UTILIDADES|            CONT_RUT|CONT_DV|       Valor|Periodo_6|Periodo_5|Periodo_4|Periodo_3|Periodo_2|Periodo_1|
+--------------------+--------------------+------------------+---------------------+--------------------+-------+------------+---------+---------+---------+---------+---------+---------+
|+++1gCGSMhlEHLJDQ...|jBIDEmi35uF/bAHr2...|             100.0|                100.0|jBIDEmi35uF/bAHr2...|      K|         1.0|     null|     null|     null|     null|     null|     null|
|+++40fb1kG88W0AEf...|bmXOLtRFjJPrLjKzQ...|             100.0|                100.0|bmXOLtRFjJPrLjKzQ...|      7|         1.0|     null|     null|     null|     null|     null|     null|
|+++aS3F42zAfvw7QX...|WdafNmzolsFbUp2HA...|             100.0|   

+--------------------+--------------------+------------------+------------+
|        RUT_SOCIEDAD|           RUT_SOCIO|PORCENTAJE_CAPITAL|       Valor|
+--------------------+--------------------+------------------+------------+
|+++1gCGSMhlEHLJDQ...|jBIDEmi35uF/bAHr2...|             100.0|         1.0|
|+++40fb1kG88W0AEf...|bmXOLtRFjJPrLjKzQ...|             100.0|         1.0|
|+++aS3F42zAfvw7QX...|WdafNmzolsFbUp2HA...|             100.0|0.1007057459|
|++/7Y0IzKUdH2J8C/...|41WRUY90glpRrq9XI...|             100.0|         1.0|
|++/p+MzwTKR1XVz6S...|+bJxBrWdS6tCHMPtN...|             100.0|        null|
|++00DcdTZAhzp008i...|aL6LEEk8P6vB8PyHu...|             100.0|         0.0|
|++0Cw0fQKv+4UABY2...|eJvYBKdvqhJ6I1vK5...|             100.0|         0.0|
|++0PaA9Gk1ZMElZZL...|Y20GxtSITZxJGWolh...|              50.0|         1.0|
|++0PaA9Gk1ZMElZZL...|VkY6uutbGR+7iTLUS...|              50.0|         1.0|
|++1jdKtXA14GodsF2...|/VVBxeWcPKI3JZxh1...|              50.0|         1.0|
|++1jdKtXA14

+--------------------+-----+
|       RUT_SOCIEDAD1|nulos|
+--------------------+-----+
|++/p+MzwTKR1XVz6S...|    1|
|++4bM0qSOip7a4an0...|    1|
|++5XoPadLnX4yG280...|    1|
|++5Zy0fiCNVQCozs/...|    1|
|++6RBZh62gpZTbwM3...|    1|
|++Alwstd+2uhLD2qz...|    1|
|++BttxIm3VHYY8A2X...|    1|
|++CnaHLm0T3aFHcyV...|    1|
|++D4ZjOwJH/LeOsC9...|    1|
|++DjunNIt3XkVA4fh...|    1|
|++EyF08j+3p7xN1Li...|    2|
|++FL4QEZEY4u/lBQI...|    1|
|++FivaO4kmKJefzpT...|    1|
|++Gmpr5y5+fTSS8k8...|    1|
|++HgREQIq0eFf0mM0...|    1|
|++IKdgAIPTibtfSbs...|    1|
|++JSQJAjHEccfP1gN...|    1|
|++JncT+Rfne2WP0Ha...|    1|
|++K39RIEkVAkdycK3...|    1|
|++KVzyU5k50YWnlIl...|    1|
+--------------------+-----+
only showing top 20 rows



[Stage 97:==============================================>      (174 + 24) / 200]

+--------------------+--------------------+------------------+-----+------------+
|        RUT_SOCIEDAD|           RUT_SOCIO|PORCENTAJE_CAPITAL|nulos|       Valor|
+--------------------+--------------------+------------------+-----+------------+
|+++1gCGSMhlEHLJDQ...|jBIDEmi35uF/bAHr2...|             100.0| null|         1.0|
|+++40fb1kG88W0AEf...|bmXOLtRFjJPrLjKzQ...|             100.0| null|         1.0|
|+++aS3F42zAfvw7QX...|WdafNmzolsFbUp2HA...|             100.0| null|0.1007057459|
|++/7Y0IzKUdH2J8C/...|41WRUY90glpRrq9XI...|             100.0| null|         1.0|
|++00DcdTZAhzp008i...|aL6LEEk8P6vB8PyHu...|             100.0| null|         0.0|
|++0Cw0fQKv+4UABY2...|eJvYBKdvqhJ6I1vK5...|             100.0| null|         0.0|
|++0PaA9Gk1ZMElZZL...|Y20GxtSITZxJGWolh...|              50.0| null|         1.0|
|++0PaA9Gk1ZMElZZL...|VkY6uutbGR+7iTLUS...|              50.0| null|         1.0|
|++1jdKtXA14GodsF2...|BNuqVX553Rgbk6Y4p...|              50.0| null|         1.0|
|++1jdKtXA14Gods

## Primera iteracion, cruce con oscuridad de personas naturales (2do paso de calculo de oscuridad y ponderacion por total de capital)

Junto con completar la data con los valores de oscuridad para sociedades completas, se agrega un ponderador. De esta forma, si una entidad tiene dos socios con oscuridad y la composicion suma diferente a 100%, se pondera proporcionalmente para que sumen 100%.

In [15]:

#agregar a la oscuridad la ponderacion adecuada

spark.sql("select RUT_SOCIEDAD,RUT_SOCIO,PORCENTAJE_CAPITAL, Valor from aux where nulos is null").createOrReplaceTempView("aux")
#spark.sql("select * from aux").show()
spark.sql("select RUT_SOCIEDAD as CONT_RUT, SUM(PORCENTAJE_CAPITAL*Valor*0.01) as othervalue from aux group by RUT_SOCIEDAD").createOrReplaceTempView("oscuridad")
#spark.sql("select * from oscuridad").show()
spark.sql("select CONT_RUT, othervalue as Value from oscuridad").createOrReplaceTempView("oscuridad")
#spark.sql("select * from oscuridad").show()

spark.sql("select oscuridad.CONT_RUT as CONT_RUT, Value, ponderador from oscuridad left join composicion on oscuridad.CONT_RUT=composicion.CONT_RUT order by oscuridad.CONT_RUT desc ").createOrReplaceTempView("oscuridad")
#spark.sql("select * from oscuridad ").show()
spark.sql("select CONT_RUT, Value/ponderador*100 as Value from oscuridad ").createOrReplaceTempView("oscuridad")
#spark.sql("select * from oscuridad order by Value desc ").show()

#spark.sql("select RUT_SOCIEDAD,RUT_SOCIO,PORCENTAJE_UTILIDADES, Valor from aux where nulos is null").createOrReplaceTempView("aux")

oscuridad_aux=spark.sql("select * from oscuridad ").toPandas()
oscuridad_aux['iterations']=0


In [16]:
oscuridad_aux

,CONT_RUT,Value,iterations
0,zzzYO2A2+lYZwSvY7O2ylw==,1.000000,0
1,zzzOmSrXr0YYFWJZ8UoGPA==,1.000000,0
2,zzzD8iwZXdSS0SA5Lw3Zbw==,1.000000,0
3,zzz6I2f5gUYdjNfBJZoaBw==,1.000000,0
4,zzye6tQDGFDGsdgTDVNGXg==,1.000000,0
...,...,...,...
1108877,++00DcdTZAhzp008i3ob6A==,0.000000,0
1108878,++/7Y0IzKUdH2J8C/F8aJA==,1.000000,0
1108879,+++aS3F42zAfvw7QXrS4nw==,0.100706,0
1108880,+++40fb1kG88W0AEf1PIgg==,1.000000,0


In [17]:
#soc = spark.sql("SELECT RUT_SOCIEDAD, PORCENTAJE_CAPITAL, Valor FROM sociedad WHERE count(RUT_SOCIO) = 1 GROU").toPandas()
#soc = spark.sql("SELECT * FROM sociedad").toPandas()
soc = spark.sql("SELECT RUT_SOCIEDAD AS CONT_RUT, COUNT(RUT_SOCIO) AS N_SOCIOS FROM sociedad GROUP BY RUT_SOCIEDAD").toPandas()

In [18]:
soc2 = soc[soc["N_SOCIOS"]==1]
soc2

,CONT_RUT,N_SOCIOS
0,+++1gCGSMhlEHLJDQDdj8g==,1
1,+++40fb1kG88W0AEf1PIgg==,1
2,+++aS3F42zAfvw7QXrS4nw==,1
3,++/7Y0IzKUdH2J8C/F8aJA==,1
4,++/p+MzwTKR1XVz6SDk3GA==,1
...,...,...
1314652,zzzYO2A2+lYZwSvY7O2ylw==,1
1314653,zn6ahFIUgBI+OKaOfRLXUg==,1
1314654,znQyEmQVaz2h+xlv6PR2VQ==,1
1314656,zvnHliUKkV9LhDvxfZSFsQ==,1


In [19]:
import pandas as pd
aux = pd.merge(soc2,oscuridad_aux, on = ["CONT_RUT"])
aux = aux.rename(columns = {"CONT_RUT": "RUT_SOCIEDAD"})
aux

,RUT_SOCIEDAD,N_SOCIOS,Value,iterations
0,+++1gCGSMhlEHLJDQDdj8g==,1,1.000000,0
1,+++40fb1kG88W0AEf1PIgg==,1,1.000000,0
2,+++aS3F42zAfvw7QXrS4nw==,1,0.100706,0
3,++/7Y0IzKUdH2J8C/F8aJA==,1,1.000000,0
4,++00DcdTZAhzp008i3ob6A==,1,0.000000,0
...,...,...,...,...
588000,zzzYO2A2+lYZwSvY7O2ylw==,1,1.000000,0
588001,zn6ahFIUgBI+OKaOfRLXUg==,1,1.000000,0
588002,znQyEmQVaz2h+xlv6PR2VQ==,1,1.000000,0
588003,zvnHliUKkV9LhDvxfZSFsQ==,1,0.120847,0


In [20]:
dataframe = oscuridad1.toPandas().rename(columns = {"CONT_RUT": "RUT_SOCIO"})
dataframe

,RUT_SOCIO,CONT_DV,Valor,Periodo_6,Periodo_5,Periodo_4,Periodo_3,Periodo_2,Periodo_1
0,MiaSIVlfzM3aarxDaafMDQ==,2,0.854984,NaN,NaN,2.0,NaN,NaN,NaN
1,g2fIxrrqgBfUvmoVGhPyyQ==,0,1.000000,NaN,NaN,NaN,NaN,NaN,NaN
2,dRt75PUFgAoouSFtbAZR9Q==,9,1.000000,NaN,NaN,NaN,NaN,NaN,NaN
3,omH0pl8oX9V7bfnkWON/MQ==,2,1.000000,NaN,NaN,NaN,NaN,NaN,NaN
4,ULdUVTTc6UnYWOzMGrK1hw==,0,1.000000,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
26619832,viofOR8KM288P+qnE5Jo0g==,6,0.540588,NaN,NaN,NaN,NaN,1.0,2.0
26619833,KEYuFS0T+76hOMrUEosJWQ==,4,0.540588,NaN,NaN,NaN,NaN,1.0,1.0
26619834,b50jVIwY/1h77GJYLw53kQ==,2,0.540588,NaN,NaN,NaN,NaN,1.0,1.0
26619835,cfIzTPUx+uoZb0UNTofZfA==,0,0.540588,NaN,NaN,NaN,NaN,1.0,3.0


In [21]:
df1 = df.toPandas()
df1

,RUT_SOCIEDAD,RUT_SOCIO,PORCENTAJE_CAPITAL,PORCENTAJE_UTILIDADES
0,lvhKR91n05T3mstWgN1uIw==,tnshXz2zZHhF/35iMHB/Fg==,100.0,100.0
1,FJ59A3rCUrWEuEI/geGBVQ==,AtxwrhZVZnI1sYNt/kS4CA==,50.0,50.0
2,U9dRqzVEukCZdQmX8/lMlA==,qlm4skAGTxB/gAmriC7fSQ==,100.0,100.0
3,hEQr1w9LgGXEeDym5Mcneg==,QP3cSviQ8v3p5Pud+MehMQ==,40.0,40.0
4,YERESpwuZelDrFKfkwF2tQ==,aefg+HqgBVthzV/Dp5b7Kw==,20.0,20.0
...,...,...,...,...
2437913,scdaD6gUx/pVe5XjW+JhSw==,fSPCB7Qr3995U+UMreravA==,10.0,10.0
2437914,N5JCHlLmHmHiatERn1xzjQ==,qxoEb4E05KK3raqIxEFZAA==,100.0,100.0
2437915,cc+Btv0AAee/zPESY8KzAw==,K6/41L29t0dsR7tJ0nV1+A==,100.0,100.0
2437916,VFsY+3e3HLke3grPu/A6uQ==,TWN/P9bHJYZ19ACveE6AYw==,33.0,33.0


In [22]:
aux2 = pd.merge(pd.merge(df1, aux, on = ["RUT_SOCIEDAD"]), dataframe, on = ["RUT_SOCIO"])
aux2 = aux2.rename(columns = {"Value":"Oscuridad_PJ","Valor":"Oscuridad_PN"})
aux2 = aux2.sort_values(by=["Oscuridad_PN"])


In [23]:
aux2[aux2["Oscuridad_PN"]>0].head(30)

,RUT_SOCIEDAD,RUT_SOCIO,PORCENTAJE_CAPITAL,PORCENTAJE_UTILIDADES,N_SOCIOS,Oscuridad_PJ,iterations,CONT_DV,Oscuridad_PN,Periodo_6,Periodo_5,Periodo_4,Periodo_3,Periodo_2,Periodo_1
13804,JRHi40pEopJdtpxJZhLPoA==,/jN5MLsN0Cr8IC7nZn97fw==,100.0,100.0,1,0.100706,0,4,0.100706,NaN,1.0,2.0,2.0,2.0,1.0
77949,fJbyO29DZQXkbY7wG9/3vg==,KVjFez+0Uy5bueDlIN9iUw==,100.0,100.0,1,0.100706,0,6,0.100706,NaN,1.0,1.0,1.0,1.0,1.0
296603,JsubyzfbVAOeVvLmDDvSBg==,4kdvn82ml9NddY6Q2kNDzA==,100.0,100.0,1,0.100706,0,4,0.100706,NaN,1.0,1.0,2.0,1.0,1.0
491065,3tQSY9bOt4Oac3P1qhLYjA==,okjXd1Ab6cIilSSBhlV/Ew==,100.0,100.0,1,0.100706,0,5,0.100706,NaN,1.0,2.0,3.0,1.0,1.0
125133,1gckLQIL40QiVHjdfxNTtw==,DKA2SBqcfCDhFa1o235OfQ==,100.0,100.0,1,0.100706,0,6,0.100706,NaN,2.0,3.0,3.0,3.0,3.0
343414,SumQ6bDVG3+FF3PJlc/RTw==,HtLQ3z3XlkjXNzqpeEG1pw==,100.0,100.0,1,0.100706,0,3,0.100706,NaN,1.0,4.0,3.0,2.0,1.0
318084,3xXEbDlY3XJ8i6WNnGAJjg==,S/1AgpgaFU4u9d8WY5UTYA==,100.0,5.0,1,0.100706,0,2,0.100706,NaN,1.0,1.0,1.0,1.0,1.0
225510,zhBfKqp8oxA/2U72AYjulA==,zQzNUXGjp6qR6AIq9bvMKA==,100.0,100.0,1,0.100706,0,9,0.100706,NaN,4.0,1.0,6.0,5.0,2.0
338726,9VTVdoC7dr3DgP8W/rKX1A==,zAbN0nptGDiM2iF2cfK31g==,100.0,100.0,1,0.100706,0,7,0.100706,NaN,4.0,2.0,1.0,1.0,1.0
510812,cQ1JEwsk8daXSjh8I5k1qg==,gAzuuGy+p+1DgpRYXRRNTQ==,100.0,100.0,1,0.100706,0,9,0.100706,NaN,1.0,1.0,1.0,3.0,2.0


In [24]:
aux2[aux2["RUT_SOCIEDAD"]== "XYtAg9XPdlsmKvL99k3t/w=="].head()
#aux2[aux2["RUT_SOCIO"]== "eK5KCE9n7ek2CyHGijVAAQ=="].head()

,RUT_SOCIEDAD,RUT_SOCIO,PORCENTAJE_CAPITAL,PORCENTAJE_UTILIDADES,N_SOCIOS,Oscuridad_PJ,iterations,CONT_DV,Oscuridad_PN,Periodo_6,Periodo_5,Periodo_4,Periodo_3,Periodo_2,Periodo_1


## Primera iteracion,resultados

Principales resultados de la primera iteracion, es decir la iteracion cero. 

In [25]:
oscuridad_aux.describe()

,Value,iterations
count,1.108882e+06,1108882.0
mean,6.716056e-01,0.0
std,3.791623e-01,0.0
min,0.000000e+00,0.0
25%,3.665689e-01,0.0
50%,8.747059e-01,0.0
75%,1.000000e+00,0.0
max,1.000000e+00,0.0


## Iteraciones subsiguientes

Se repite el proceso iterativo para asi completar mas sociedades. El parametro *iter* hara referencia al numero de iteraciones luego de la iteracion 0.

In [26]:
iter=6

In [27]:
for iteration in range(1,iter):
    
    spark.sql("select * from sociedad left join oscuridad on sociedad.RUT_SOCIO=oscuridad.CONT_RUT order by sociedad.RUT_SOCIEDAD asc").createOrReplaceTempView("sociedad")
    spark.sql("select RUT_SOCIEDAD, RUT_SOCIO, PORCENTAJE_CAPITAL, CASE WHEN Valor is null THEN Value ELSE Valor END AS Valor from sociedad").createOrReplaceTempView("sociedad")
    spark.sql("select RUT_SOCIEDAD, RUT_SOCIO, PORCENTAJE_CAPITAL, Valor from sociedad").createOrReplaceTempView("sociedad")
    spark.sql("select RUT_SOCIEDAD as RUT_SOCIEDAD1, count(*) as nulos from sociedad where Valor is null group by RUT_SOCIEDAD order by RUT_SOCIEDAD ASC").createOrReplaceTempView("aux")
    spark.sql("select RUT_SOCIEDAD,RUT_SOCIO,PORCENTAJE_CAPITAL, nulos, Valor from sociedad left join aux on sociedad.RUT_SOCIEDAD=aux.RUT_SOCIEDAD1 order by RUT_SOCIEDAD asc ").createOrReplaceTempView("aux")
    spark.sql("select RUT_SOCIEDAD,RUT_SOCIO,PORCENTAJE_CAPITAL, Valor from aux where nulos is null").createOrReplaceTempView("aux")
    spark.sql("select RUT_SOCIEDAD as CONT_RUT, SUM(PORCENTAJE_CAPITAL*Valor*0.01) as othervalue from aux group by RUT_SOCIEDAD").createOrReplaceTempView("oscuridad")
    spark.sql("select CONT_RUT, othervalue as Value from oscuridad").createOrReplaceTempView("oscuridad")
    
    spark.sql("select oscuridad.CONT_RUT as CONT_RUT, Value,ponderador from oscuridad left join composicion on oscuridad.CONT_RUT=composicion.CONT_RUT order by oscuridad.CONT_RUT asc ").createOrReplaceTempView("oscuridad")
    #spark.sql("select * from oscuridad ").show()
    spark.sql("select CONT_RUT, Value/ponderador*100 as Value from oscuridad order by Value desc ").createOrReplaceTempView("oscuridad")
    #spark.sql("select * from oscuridad ").show()    
    oscuridad=spark.sql("select * from oscuridad ").toPandas()
    oscuridad['iterations']=iteration

    oscuridad=oscuridad.merge(oscuridad_aux, on = "CONT_RUT", how = "left")
    oscuridad['iterations']=oscuridad[["iterations_x", "iterations_y"]].min(axis=1)
    oscuridad = oscuridad.rename(columns={'Value_x': 'Value'})
    oscuridad=oscuridad[['CONT_RUT','Value','iterations']]
    oscuridad_aux=oscuridad.iloc[:,:]
    print(oscuridad_aux.describe())

              Value    iterations
count  1.119135e+06  1.119135e+06
mean   6.726528e-01  9.161540e-03
std    3.785106e-01  9.527651e-02
min    0.000000e+00  0.000000e+00
25%    3.665689e-01  0.000000e+00
50%    8.747059e-01  0.000000e+00
75%    1.000000e+00  0.000000e+00
max    1.000000e+00  1.000000e+00


              Value    iterations
count  1.119802e+06  1.119802e+06
mean   6.727747e-01  1.034736e-02
std    3.784535e-01  1.069186e-01
min    0.000000e+00  0.000000e+00
25%    3.665689e-01  0.000000e+00
50%    8.747059e-01  0.000000e+00
75%    1.000000e+00  0.000000e+00
max    1.000000e+00  2.000000e+00


              Value    iterations
count  1.119868e+06  1.119868e+06
mean   6.727892e-01  1.052356e-02
std    3.784483e-01  1.093510e-01
min    0.000000e+00  0.000000e+00
25%    3.665689e-01  0.000000e+00
50%    8.747059e-01  0.000000e+00
75%    1.000000e+00  0.000000e+00
max    1.000000e+00  3.000000e+00


              Value    iterations
count  1.119868e+06  1.119868e+06
mean   6.727892e-01  1.052356e-02
std    3.784483e-01  1.093510e-01
min    0.000000e+00  0.000000e+00
25%    3.665689e-01  0.000000e+00
50%    8.747059e-01  0.000000e+00
75%    1.000000e+00  0.000000e+00
max    1.000000e+00  3.000000e+00


              Value    iterations
count  1.119868e+06  1.119868e+06
mean   6.727892e-01  1.052356e-02
std    3.784483e-01  1.093510e-01
min    0.000000e+00  0.000000e+00
25%    3.665689e-01  0.000000e+00
50%    8.747059e-01  0.000000e+00
75%    1.000000e+00  0.000000e+00
max    1.000000e+00  3.000000e+00


## Resultados y guardado en archivo csv

Luego de estas iteraciones, se obtienen dos outputs. En primer lugar una tabla actualizada de sociedades y socios con cada valor calculado (o no) de la oscuridad de sus socios. 
En segundo lugar la oscuridad calculada para sociedades.

In [28]:
sociedad=spark.sql("select * from sociedad").toPandas()

In [29]:
# Se guardan los archivos finales.
sociedad.to_csv('../../artefactos/Oscuridad/Sociedades_oscuridad_actualizada.csv', index=False)
oscuridad_aux.to_csv('../../artefactos/Oscuridad/Sociedades_oscuridad_completa.csv', index=False)


In [ ]:
df=spark.sql("select * from sociedad")
df.write.mode('overwrite').format("parquet").save("abfs://data@datalakesii.dfs.core.windows.net/DatoOrigen/lr-629/UtilBajo/intermedia/oscuridad_sociedad")

